In [35]:
using CSV
using DataFrames

df = CSV.read("uwcourses.csv", DataFrame)

list_courses = []
for col in eachcol(df)
    for course in col
        if ismissing(course) == true
            continue
        end
        
        push!(list_courses, course)
    end
end
courses = collect(Set(list_courses)) 

println(df)

91×3 DataFrame
 Row │ computer-science       data-science            economics                         
     │ String31               String31?               String?                           
─────┼──────────────────────────────────────────────────────────────────────────────────
   1 │ CS/MATH 240            MATH 221                MATH 221
   2 │ CS/ECE 252             MATH 222                MATH 171
   3 │ CS 300                 STAT 240                MATH 217
   4 │ CS/ECE 354             STAT 340                MATH 211
   5 │ CS 400                 CS 220                  MATH 213
   6 │ MATH 221               CS 300                  MATH 211
   7 │ MATH 222               CS 320                  ECON 205
   8 │ MATH 171               LIS 461                 ECON 310
   9 │ MATH 217               MATH 217                ECON 400
  10 │ MATH 222               ECE/ISYE 570            ECON 410
  11 │ MATH 320               CS/ECE/ME 532           MATH/STAT 309
  12 │ MATH 340     

In [32]:
using CSV
using DataFrames

# Load the CSV file into a DataFrame
courses_df = CSV.read("uwcourses.csv", DataFrame)

# Convert all columns to a uniform type of Vector{String} to ensure compatibility
for col in names(courses_df)
    courses_df[!, col] = convert(Vector{String}, string.(courses_df[!, col]))
end

# Define a function to clean the input and extract the numerical part of the course name
function extract_course_number(course_name)
    if typeof(course_name) == String
        # Clean up the input by trimming whitespace
        cleaned_course_name = strip(course_name)
        # Match the first number in the string
        match_obj = match(r"\d+", cleaned_course_name)
        return match_obj !== nothing ? parse(Int, match_obj.match) : missing
    else
        return missing
    end
end

# Create a new DataFrame to store the extracted course numbers
course_numbers_df = DataFrame()

# Iterate over each column and extract the course numbers
for col in names(courses_df)
    # Extract course numbers for each value in the current column
    extracted_numbers = [extract_course_number(course_name) for course_name in courses_df[!, col]]
    # Add the extracted numbers as a new column in the new DataFrame
    course_numbers_df[!, col] = extracted_numbers
end

println(course_numbers_df)

91×3 DataFrame
 Row │ computer-science  data-science  economics 
     │ Int64             Int64?        Int64?    
─────┼───────────────────────────────────────────
   1 │              240           221        221
   2 │              252           222        171
   3 │              300           240        217
   4 │              354           340        211
   5 │              400           220        213
   6 │              221           300        211
   7 │              222           320        205
   8 │              171           461        310
   9 │              217           217        400
  10 │              222           570        410
  11 │              320           532        309
  12 │              340           539        431
  13 │              341           540        311
  14 │              375           656        324
  15 │              309           521        340
  16 │              311           535        101
  17 │              324           361        102
  

In [37]:
using JuMP, GLPK

# Define the model with the GLPK solver
model = Model(GLPK.Optimizer)

# Create a binary variable for each course
@variable(model, course_taken[courses], Bin)

@constraint(model, course_taken["CS 200"] + course_taken["CS/MATH 240"] + course_taken["CS/ECE 252"] + course_taken["CS 300"] + course_taken["CS/ECE 354"] + course_taken["CS 400"] >= 6)

@constraint(model, course_taken["MATH 221"] + course_taken["MATH 222"] >= 2)

@constraint(model, course_taken["MATH 320"] + course_taken["MATH 340"] + course_taken["MATH 341"] + course_taken["MATH 375"] >= 1)

@constraint(model, course_taken["STAT/MATH 309"] + course_taken["STAT 311"] + course_taken["STAT 324"] + course_taken["STAT 333"] + course_taken["STAT 340"] + course_taken["STAT 371"] + course_taken["MATH/STAT 431"] + course_taken["MATH 531"] >= 1)

@constraint(model, course_taken["MATH 221"] + course_taken["MATH 217"] + course_taken["MATH 222"] >= 3)

@constraint(model, course_taken["STAT 240"] + course_taken["STAT 340"] + course_taken["CS 220"] + course_taken["CS 300"] + course_taken["CS 320"] >= 5)

@constraint(model, course_taken["CS 400"] + course_taken["CS 412"] + course_taken["CS/STAT 471"] + course_taken["CS/MATH 513"] + course_taken["CS/MATH 514"] + course_taken["CS/ECE/ISYE 524"] + course_taken["CS 544"] + course_taken["CS 564"] + course_taken["CS 565"] + course_taken["CS/BMI 576"] >= 1)

@constraint(model, course_taken["CS/ECE/ME 532"] + course_taken["CS/ECE/ME 539"] + course_taken["CS 540"] + course_taken["GEN BUS 656"] + course_taken["ISYE 521"] + course_taken["MATH 535"] + course_taken["PHYSICS 361"] + course_taken["STAT 451"] + course_taken["STAT 453"] >= 1)

@constraint(model, course_taken["MATH 221"] + course_taken["MATH 171"] + course_taken["MATH 217"] + course_taken["MATH 211"] + course_taken["MATH 213"] + course_taken["ECON 205"] >= 2)

@constraint(model, course_taken["ECON 310"] + course_taken["ECON 400"] + course_taken["ECON 410"] + course_taken["MATH/STAT 309"] + course_taken["STAT 311"] + course_taken["STAT 324"] + course_taken["STAT 340"] >= 1)

@constraint(model, course_taken["ECON 101"] + course_taken["ECON 102"] + course_taken["ECON 111"] >= 2)

@constraint(model, course_taken["ECON 301"] + course_taken["ECON 302"] + course_taken["ECON 311"] + course_taken["ECON 312"] >= 2)

# Objective: Minimize the total number of courses
@objective(model, Min, sum(course_taken))

# Solve the optimization problem
optimize!(model)

# Check if a solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found. Courses to enroll:")
    for course in courses
        if value(course_taken[course]) > 0.5
            println(course)
        end
    end
else
    println("No optimal solution found.")
end


Optimal solution found. Courses to enroll:
CS/ECE/ME 539
ECON 301
ECON 102
ECON 311
STAT 240
MATH 222
ECON 111
CS 200
CS 400
MATH 320
CS 220
CS/MATH 240
CS 320
CS 300
CS/ECE 252
MATH 221
STAT 340
CS/ECE 354
MATH 217
